In [75]:
import pandas as pd
import os
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, \
                             AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from merf import MERF
from library import *

import itertools

np.random.seed(42)

df = pd.read_csv('WLOAD_notch_bp_avg_mastoid_annotated_ica_fft.csv')

df['trial_id'] = df['trial_id'].apply(lambda x: x[2:-2])


# -1 trials are not used [first three for nback and/or rest, etc.]
df = df[df['workload'] != '-1']

# 'True'/'False' is used by stroop, whereas 0,1,2,3 are used for nback and rotation
df['workload'] = df['workload'].apply(lambda x: '3.0' if x == 'True' else '0.0' if x == "False" else x)
df['workload'] = df['workload'].astype(float).astype(int)

for i, row in df.iterrows():
    if row['workload'] == 1:        
        df['workload'][i] = 0
    elif row['workload'] == 2 or row['workload'] == 3:
        df['workload'][i] = 1

# DROP ROTATION FOR NOW.

all_results = {'nback': [], 'stroop': [], 'all': []}
trial_types = ['nback', 'stroop']

# Generate all combinations of two trial types
combinations = list(itertools.combinations(trial_types, 2))

# Add combinations to the results dictionary
for combo in combinations:
    all_results[' + '.join(combo)] = []

for key in ['nback', 'stroop']:
    print(f"{key} - {df[df['trialtype'] == key].shape[0]}")

size_stroop = df[df['trialtype'] == 'stroop'].shape[0]

for option in all_results.keys():
    
    if option == 'all':
        subdf = df.copy()
    elif option in trial_types:
        subdf = df[df['trialtype'] == option]
    else:
        # For combinations, concatenate the dataframes
        types = option.split(' + ')
        subdf = pd.concat([df[df['trialtype'] == t] for t in types])

    freqd = produce_freq_bands(subdf)
    X_train, X_test, y_train, y_test = split_data(freqd)
    X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
        
    results = classify(X_train_scaled, X_test_scaled, y_train, y_test)
    all_results[option].append(results)
    
results = pd.concat({key: pd.concat(val).T for key, val in all_results.items()}, axis=1)
results

nback - 2933
stroop - 2759
(2137, 20) - (2137,) - (796, 20) - (796,)
[LightGBM] [Info] Number of positive: 1162, number of negative: 975
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 2137, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.543753 -> initscore=0.175460
[LightGBM] [Info] Start training from score 0.175460
(2009, 20) - (2009,) - (750, 20) - (750,)
[LightGBM] [Info] Number of positive: 996, number of negative: 1013
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000176 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5100
[LightGBM] [Info] Number of data points in the train set: 2009, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pa

,nback,stroop,all,nback + stroop
svc,0.582610,0.541035,0.541767,0.571697
log_reg,0.494962,0.524316,0.506786,0.514718
rf,0.811558,0.622612,0.660661,0.721833
knn,0.722235,0.563720,0.586736,0.645436
ann,0.752793,0.572552,0.593556,0.665384
gbm,0.766330,0.581159,0.634233,0.669280
lightgbm,0.822851,0.579083,0.647697,0.701002
xgboost,0.816564,0.576610,0.656576,0.712456


In [69]:
from sklearn.utils import resample
def undersample_dataset(X, y, size, random_state=42):
    X_undersampled, y_undersampled = resample(X, y, replace=False, n_samples=size, random_state=random_state)
    return X_undersampled, y_undersampled

Rotation	xgboost	   0.6287
Nback	    lightgbm   0.8229
Stroop	    rf	       0.6213
All	        rf	       0.6719

In [24]:

def produce_freq_bands(df):

    # Define the frequency bands
    freq_bands = {
        "Delta": (0.5, 4),
        "Theta": (4, 8),
        "Alpha": (8, 14),
        "Beta":  (14, 30),
        "Gamma": (30, 45)
    }

    # Extract only the columns with frequency data
    freq_data = df.filter(like="freq")

    binned_df = pd.DataFrame()

    for ch in range(4):  # Assuming you have 4 channels: ch_0, ch_1, ch_2, ch_3
        for band, (f_min, f_max) in freq_bands.items():
            # Filter columns for the current channel and band
            cols = [col for col in freq_data.columns 
                    if f"ch_{ch}_freq_" in col 
                    and f_min <= float(col.split("_")[3][:-2]) <= f_max]
            
            # Aggregate the columns and add to the binned dataframe
            if cols:
                binned_df[f"ch_{ch}_{band}"] = freq_data[cols].mean(axis=1)
            else:
                binned_df[f"ch_{ch}_{band}"] = np.nan

    # Add back non-frequency columns to the binned dataframe
    for col in ['trialtype', 'correct', 'workload', 'pid', 'trial_id', 'filename']:
        binned_df[col] = df[col]

    df = binned_df
    return df 

within subjects test; make sure to separate data into EITHER test/train based on which file produced the data. technically could get more specific with nback task, but this is generalizable.  

In [66]:
from sklearn.model_selection import GroupShuffleSplit

def split_data(df):

    X = df.drop(columns=['trialtype', 'correct', 'workload', 'pid', 'trial_id', 'filename'])
    y = df['workload']
    pid = df['pid']

    # Initial split based on pid
    X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

    # Iterate over each unique pid
    for unique_pid in pid.unique():

        # Filter data for the current pid
        pid_data = df[df['pid'] == unique_pid]
        
        X_pid = pid_data.drop(columns=['trialtype', 'correct', 'workload', 'pid', 'trial_id', 'filename'])
        y_pid = pid_data['workload']
        group_pid = pid_data['filename']
        
        # Use GroupShuffleSplit to split data based on filename for the current pid
        gss = GroupShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
        train_idx, test_idx = next(gss.split(X_pid, y_pid, groups=group_pid))
        
        X_train_list.append(X_pid.iloc[train_idx])
        X_test_list.append(X_pid.iloc[test_idx])
        y_train_list.append(y_pid.iloc[train_idx])
        y_test_list.append(y_pid.iloc[test_idx])

    # Concatenate the results to get the final train and test sets
    X_train = pd.concat(X_train_list)
    X_test = pd.concat(X_test_list)
    y_train = pd.concat(y_train_list)
    y_test = pd.concat(y_test_list)

    print(f"{X_train.shape} - {y_train.shape} - {X_test.shape} - {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [28]:
from sklearn.preprocessing import StandardScaler

def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled


In [61]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb

def classify(X_train_scaled, X_test_scaled, y_train, y_test):
    models = {  
                'svc': SVC(), 
                'log_reg': LogisticRegression(max_iter=10000), 
                'rf': RandomForestClassifier(), 
                'knn': KNeighborsClassifier(n_neighbors=5),  # Start with 5 neighbors, 
                'ann': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=10000),  # Simple 2-layer feedforward network,
                'gbm': GradientBoostingClassifier(), 
                'lightgbm': lgb.LGBMClassifier(), 
                'xgboost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
            }

    results = {model: None for model in models.keys()}

    for model_name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        f1 = f1_score(y_test, y_pred, average='macro')    
        results[model_name] = f1

    results = pd.Series(results)
    return results


try participant based split

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import lightgbm as lgb
import xgboost as xgb
import pandas as pd

# Get unique subjects
unique_subjects = df['pid'].unique()

# Initialize a DataFrame to store F1 scores for each participant and model
f1_scores_df = pd.DataFrame(index=unique_subjects, columns=['svc', 'rf', 'dummy'])

for p in unique_subjects:
    # Filter the dataset based on the subjects in each set
    train_data = df[df['pid'] != p]
    test_data = df[df['pid'] == p]

    # Extract features (brain data) and target variable
    X_train = train_data.filter(like="ch", axis=1)
    y_train = train_data['workload']
    y_train = y_train.replace({0: 0, 3: 1})
    

    X_test = test_data.filter(like="ch", axis=1)
    y_test = test_data['workload']
    y_test = y_test.replace({0: 0, 3: 1})

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
      
    models = {
        'svc': SVC(), 
        'rf': RandomForestClassifier(), 
        'dummy': DummyClassifier(strategy="most_frequent"), 
        'knn': KNeighborsClassifier(n_neighbors=5),  # Start with 5 neighbors, 
        'ann': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=10000),  # Simple 2-layer feedforward network,
        'gbm': GradientBoostingClassifier(), 
        #'lightgbm': lgb.LGBMClassifier(), 
        'xgboost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')  # To avoid warning
    }

    for modelname in models.keys():
        model = models[modelname]
        model.fit(X_train, y_train)
        
        preds = model.predict(X_test)    
        
        # Calculate macro average F1 score and store in the DataFrame
        f1_macro = f1_score(y_test, preds, average='macro')
        f1_scores_df.loc[p, modelname] = f1_macro

        if modelname == 'rf':
            print(classification_report(y_test, preds))

print(f1_scores_df)


              precision    recall  f1-score   support

           0       0.53      0.57      0.55       367
           1       0.50      0.45      0.47       345

    accuracy                           0.51       712
   macro avg       0.51      0.51      0.51       712
weighted avg       0.51      0.51      0.51       712

              precision    recall  f1-score   support

           0       0.49      0.82      0.61       320
           1       0.65      0.28      0.39       381

    accuracy                           0.53       701
   macro avg       0.57      0.55      0.50       701
weighted avg       0.58      0.53      0.49       701

              precision    recall  f1-score   support

           0       0.59      0.44      0.50       358
           1       0.53      0.67      0.59       336

    accuracy                           0.55       694
   macro avg       0.56      0.56      0.55       694
weighted avg       0.56      0.55      0.55       694

              preci

ensemble method

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, classification_report
import pandas as pd

# Get unique subjects
unique_subjects = df['pid'].unique()

# Initialize a DataFrame to store F1 scores for each participant and model
f1_scores_df = pd.DataFrame(index=unique_subjects, columns=['stacked', 'rf', 'dummy'])

# Outer loop: Participants reserved for testing
for test_p in unique_subjects:
    
    # Filter the dataset for the test participant
    test_data = df[df['pid'] == test_p]
    X_test = test_data.filter(like="ch", axis=1)
    y_test = test_data['workload']

    # Inner loop: Participants reserved for training the meta-model
    for meta_p in unique_subjects:
        
        if meta_p == test_p:
            continue  # Skip if meta participant is same as test participant

        # Split data
        meta_data = df[df['pid'] == meta_p]
        train_data = df[(df['pid'] != test_p) & (df['pid'] != meta_p)]

        # Extract features and target variable
        X_train = train_data.filter(like="ch", axis=1)
        y_train = train_data['workload']

        X_meta = meta_data.filter(like="ch", axis=1)
        y_meta = meta_data['workload']

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_meta = scaler.transform(X_meta)
        X_test = scaler.transform(X_test)

        models = { 
                    'rf': RandomForestClassifier(),                 
                    'knn': KNeighborsClassifier(n_neighbors=5),  # Start with 5 neighbors, 
                    'ann': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=10000),  # Simple 2-layer feedforward network,
                    'gbm': GradientBoostingClassifier(),                     
                }
        meta_features = []

        # Train base models and get predictions (meta-features) for meta-data
        for modelname in models.keys():
            model = models[modelname]
            model.fit(X_train, y_train)
            preds = model.predict_proba(X_meta)[:, 1]  # Use predicted probabilities as meta-features
            meta_features.append(preds)

        # Prepare meta-feature matrix for meta-model training
        X_meta_features = np.column_stack(meta_features)

        # Train the meta-model
        meta_model = LogisticRegression(max_iter=10000)
        meta_model.fit(X_meta_features, y_meta)

        # Predict using base models on test data and use meta-model for final prediction
        test_meta_features = []
        for modelname in models.keys():
            model = models[modelname]
            preds = model.predict_proba(X_test)[:, 1]
            test_meta_features.append(preds)
        
        X_test_meta_features = np.column_stack(test_meta_features)
        final_preds = meta_model.predict(X_test_meta_features)

        # Calculate F1 score for the stacked model
        f1_stacked = f1_score(y_test, final_preds, average='macro')
        f1_scores_df.loc[test_p, 'stacked'] = f1_stacked

        # Evaluate base models (as before)
        for modelname in models.keys():
            model = models[modelname]
            preds = model.predict(X_test)
            f1 = f1_score(y_test, preds, average='macro')
            f1_scores_df.loc[test_p, modelname] = f1

print(f1_scores_df)

           stacked        rf     dummy       svc
0cbaf29f   0.38843  0.267327   0.38843  0.388430
239022dd  0.293478  0.368932  0.368932  0.368932
5dbdbc27     0.344  0.322314     0.344  0.344000
67da2976  0.373913  0.287129  0.373913  0.373913
7f2c0be7  0.335484  0.331169  0.335484  0.335484
86a4a2ac  0.397849  0.253333  0.397849  0.397849
992cdda3     0.344  0.322314     0.344  0.344000
b07e0a64  0.343511  0.322835  0.343511  0.343511
b86bd471  0.330579  0.330579  0.336066  0.336066
